Installing dependencies

In [8]:
pip install ta backtesting bokeh==2.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.6.2
    Uninstalling bokeh-3.6.2:
      Successfully uninstalled bokeh-3.6.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 1.6.0 requires bokeh<3.7.0,>=3.5.0, but you have bokeh 2.4.3 which is incompatible.
holoviews 1.20.0 requires bokeh>=3.1, but you have bokeh 2.4.3 which is incompatible.


importing dependencies

In [9]:
import requests
import pandas as pd
import ta
from backtesting import Backtest, Strategy

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Importing OHLC data for backtesting using Polygon.io API

In [11]:
API_KEY = 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ'

def get_xauusd_data(api_key, multiplier=5, timespan="minute", from_date="2024-08-30", to_date="2025-01-30"):
    url = f"https://api.polygon.io/v2/aggs/ticker/C:XAUUSD/range/{multiplier}/{timespan}/{from_date}/{to_date}"

    params = {
        "apiKey": 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ',
        "limit": 100000
    }

    all_data = []
    next_url = url

    while next_url:
        response = requests.get(next_url, params=params)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])

            next_url = data.get("next_url")
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    if all_data:
        df = pd.DataFrame(all_data)
        df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
        return df[['timestamp', 'o', 'h', 'l', 'c', 'v']]
    else:
        return None

xauusd_data_5min = get_xauusd_data(API_KEY)
if xauusd_data_5min is not None:
    print(xauusd_data_5min.head())

            timestamp        o        h        l        c   v
0 2024-08-30 00:00:00  2519.17  2519.17  2518.12  2518.57  13
1 2024-08-30 00:05:00  2518.12  2518.99  2518.12  2518.49  13
2 2024-08-30 00:10:00  2518.44  2518.46  2518.08  2518.37  13
3 2024-08-30 00:15:00  2518.28  2518.57  2518.08  2518.08  13
4 2024-08-30 00:20:00  2518.57  2519.02  2518.33  2518.86  13


Renaming columns

In [12]:
df = xauusd_data_5min.rename(columns={'timestamp': 'Time', 'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v' : 'Volume'})
df

,Time,Open,High,Low,Close,Volume
0,2024-08-30 00:00:00,2519.17,2519.17,2518.12,2518.57,13
1,2024-08-30 00:05:00,2518.12,2518.99,2518.12,2518.49,13
2,2024-08-30 00:10:00,2518.44,2518.46,2518.08,2518.37,13
3,2024-08-30 00:15:00,2518.28,2518.57,2518.08,2518.08,13
4,2024-08-30 00:20:00,2518.57,2519.02,2518.33,2518.86,13
...,...,...,...,...,...,...
29263,2025-01-30 23:35:00,2793.15,2793.67,2792.10,2793.42,15
29264,2025-01-30 23:40:00,2793.37,2794.17,2792.80,2793.27,15
29265,2025-01-30 23:45:00,2794.04,2795.60,2792.80,2794.52,14
29266,2025-01-30 23:50:00,2794.95,2795.80,2794.04,2794.69,14


adding RSI and EMA column

In [13]:
df.set_index('Time', inplace = True)


df['Rsi'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

df['Ema9'] = ta.trend.EMAIndicator(df['Close'], window=9).ema_indicator()


df.dropna(inplace=True)

df

,Open,High,Low,Close,Volume,Rsi,Ema9
Time,,,,,,,
2024-08-30 01:05:00,2517.32,2517.32,2515.39,2515.39,13,20.912951,2517.260224
2024-08-30 01:10:00,2515.76,2515.83,2515.14,2515.36,13,20.771942,2516.880179
2024-08-30 01:15:00,2515.24,2517.61,2515.24,2517.61,13,48.706455,2517.026144
2024-08-30 01:20:00,2517.04,2518.33,2516.21,2517.86,13,50.782899,2517.192915
2024-08-30 01:25:00,2517.96,2519.13,2517.82,2518.97,13,58.764600,2517.548332
...,...,...,...,...,...,...,...
2025-01-30 23:35:00,2793.15,2793.67,2792.10,2793.42,15,50.511160,2793.451482
2025-01-30 23:40:00,2793.37,2794.17,2792.80,2793.27,15,49.891191,2793.415186
2025-01-30 23:45:00,2794.04,2795.60,2792.80,2794.52,14,54.863046,2793.636148


Backtesting strategy for given data using backtesting.py

In [14]:
class RSI_EMA_Strategy(Strategy):
    pip_size = 0.1
    percent_risk = 0.1


    def init(self):
        pass

    def next(self):
        if self.position:
            return

        equity_to_use = self._broker.equity
        lot_size = (equity_to_use * self.percent_risk) / (self.pip_size * self.sl_pips)
        units = int(lot_size)

        if units <= 0:
            return

        position_size = units

        if (self.data.Close[-1] > self.data.Open[-1] and
            self.data.Rsi[-1] < self.rsi_low and
            self.data.Close[-1] < self.data.Ema9[-1] - self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price - self.sl_pips * self.pip_size
            take_profit = entry_price + self.target_pips * self.pip_size

            if stop_loss < entry_price < take_profit:
                self.buy(size=position_size, sl=stop_loss, tp=take_profit)

        elif (self.data.Close[-1] < self.data.Open[-1] and
              self.data.Rsi[-1] > self.rsi_high and
              self.data.Close[-1] > self.data.Ema9[-1] + self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price + self.sl_pips * self.pip_size
            take_profit = entry_price - self.target_pips * self.pip_size

            if take_profit < entry_price < stop_loss:
                self.sell(size=position_size, sl=stop_loss, tp=take_profit)

target_pips_list = [15]
rsi_low_list = [20,30]
rsi_high_list = [60,70,80]
sl_pips_list = [15]
pip_difference_list = [10,15]

results = []

for target_pips in target_pips_list:
    for rsi_low in rsi_low_list:
        for rsi_high in rsi_high_list:
            for sl_pips in sl_pips_list:
                for pip_difference in pip_difference_list:

                    RSI_EMA_Strategy.target_pips = target_pips
                    RSI_EMA_Strategy.sl_pips = sl_pips
                    RSI_EMA_Strategy.rsi_low = rsi_low
                    RSI_EMA_Strategy.rsi_high = rsi_high
                    RSI_EMA_Strategy.pip_difference = pip_difference

                    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin=0.005)
                    stats = bt.run()

                    results.append({
                        'target_pips': target_pips,
                        'sl_pips': sl_pips,
                        'rsi_low': rsi_low,
                        'rsi_high': rsi_high,
                        'pip_difference': pip_difference,
                        'Final Equity': stats['Equity Final [$]'],
                        '# Trades': stats['# Trades'],
                        'Win Rate [%]': stats['Win Rate [%]'],
                        'Sharpe Ratio': stats['Sharpe Ratio'],
                        'Return': stats['Return [%]'],
                        'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
                        'Avg. Drawdown [%]': stats['Avg. Drawdown [%]'],
                        'Worst Trade [%]': stats['Worst Trade [%]'],
                        'Best Trade [%]': stats['Best Trade [%]']
                    })

results_df = pd.DataFrame(results)


try:
    best_result = results_df.loc[results_df['Final Equity'].idxmax()]

    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin= 0.005)
    bt.run(target_pips=best_result['target_pips'], rsi_low=best_result['rsi_low'], rsi_high=best_result['rsi_high'], pip_difference=best_result['pip_difference'],sl_pips = best_result['sl_pips'])
    bt.plot()
except Exception as e:
    print(f"Error during plotting: {e}")

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:125: UserWarning: Data contains too many candlesticks to plot; downsampling to '30min'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


Printing parameters for best result

In [15]:
best_result

,0
target_pips,15.000000
sl_pips,15.000000
rsi_low,20.000000
rsi_high,60.000000
pip_difference,10.000000
Final Equity,80877.850000
# Trades,846.000000
Win Rate [%],51.418440
Sharpe Ratio,0.000469
Return,708.778500
